In [1]:
import pandas as pd
import numpy as np

Data Cleaning and Import, Summary Statistics

In [4]:
df = pd.read_csv('UCI_CC_defaulting.csv')
df.head()

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


Research Question: Is it possible to fairly predict the risk of an indvidual defaulting on their credit card?
Hypotheses?: young, ummarried people that are college or lower level of education (irresponsible when younger?), take into account the fact that this is data from Tawian not USA
Evaluation Metrics: 
- calibrationl goal to improve this
- statisical parity; at least evaluate this